In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re
import tensorflow
import nltk

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
#from tensorflow.keras.utils.np_utils import to_categorical

In [ ]:
!pip install sastrawi

In [ ]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')

#augment the stopwords with nonstandard twitter words
stopwords_set = set(stopwords.words("indonesian"))
stopwords_aug = {"ya","yak","iya","yg","ga","gak","gk","udh","sdh","udah","dah","nih","ini","deh","sih","dong","donk",
                 "sm","knp","utk","yaa","tdk","gini","gitu","bgt","gt","nya","kalo","cb","jg","jgn","gw","ge",
                 "sy","min","mas","mba","mbak","pak","kak","trus","trs","bs","bisa","aja","saja","no",
                 "w","g","gua","gue","emang","emg","wkwk","dr","kau","dg","gimana","apapun","apa",
                 "klo","yah","banget","pake","terus","krn","jadi","jd","mu","ku","si","hehe",
                 "tp","pa","lu","lo","lw","tw","tau","karna","kayak","ky","lg","untuk","tuk","dg","dgn"}
stopwords_all = stopwords_set.union(stopwords_aug)

In [ ]:
from matplotlib import pyplot
%matplotlib inline

def plot_model_history(model_history, width=15, height=6):

    # summarize history for accuracy
    pyplot.figure(figsize=(width,height))
    pyplot.plot(history.history['accuracy'])
    pyplot.plot(history.history['val_accuracy'])
    pyplot.title('model accuracy')
    pyplot.ylabel('accuracy')
    pyplot.xlabel('epoch')
    pyplot.legend(['train', 'test'], loc='upper left')
    pyplot.show()

    # summarize history for loss
    pyplot.figure(figsize=(width,height))
    pyplot.plot(history.history['loss'])
    pyplot.plot(history.history['val_loss'])
    pyplot.title('model loss')
    pyplot.ylabel('loss')
    pyplot.xlabel('epoch')
    pyplot.legend(['train', 'test'], loc='upper left')
    pyplot.show()

In [ ]:
##!wget https://raw.githubusercontent.com/project303/Deep-Learning-201/master/dataset/sentiment.csv

In [ ]:
!wget https://raw.githubusercontent.com/project303/dataset/master/Twitter.csv

In [ ]:
df = pd.read_csv('Twitter.csv', sep='|')

# Ambil field text dn sentiment
df = df[['text','sentiment']]


In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
df.head(10)

In [ ]:
#data = data[data.sentiment != "Neutral"]

dataset = df[df.sentiment != "Netral"]

#dataset['text'] =  [re.sub('[^a-zA-z0-9\s]','', str(x)) for x in dataset['text']]

text_column = dataset['text']
clean_text = text_column.str.replace(r'[^a-zA-Z\s]', '', regex=True).str.strip()
clean_text = clean_text.str.replace('rt',' ')
clean_text = clean_text.str.lower()
#clean_text = clean_text.str.replace(stopwords_all, '')

dataset['clean_text'] = clean_text


In [ ]:
dataset.head(10)

In [ ]:
print(dataset[ dataset['sentiment'] == 'Positif'].size)
print(dataset[ dataset['sentiment'] == 'Negatif'].size)

In [ ]:
max_fatures = 10000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(dataset['clean_text'].values)
#dataset_text = tokenizer.texts_to_sequences(dataset['clean_text'].values)
#dataset_text = pad_sequences(dataset_text, maxlen=100)
#dataset_label = pd.get_dummies(dataset['sentiment']).values

In [ ]:
train_data_, test_data_, train_label_, test_label_ = train_test_split(dataset['clean_text'], dataset['sentiment'], test_size = 0.3, random_state = 42)

In [ ]:
train_data_.head()

In [ ]:
test_label_.head()

In [ ]:
train_data = tokenizer.texts_to_sequences(train_data_.values)
train_data = pad_sequences(train_data, maxlen=100)
train_label = pd.get_dummies(train_label_).values

test_data = tokenizer.texts_to_sequences(test_data_.values)
test_data = pad_sequences(test_data, maxlen=100)
test_label = pd.get_dummies(test_label_).values

In [ ]:
test_label

In [ ]:
print(train_data.shape,train_label.shape)
print(test_data.shape,test_label.shape)

In [ ]:
train_data_.head()

In [ ]:
train_data[:5]

In [ ]:
word_index = tokenizer.word_index
print(word_index)

## Create Model

In [ ]:
max_features = 10000
max_length = 100
embedding_size = 32
lstm_output_size = 196

model = Sequential()
model.add(Embedding(max_fatures, embedding_size, input_shape=(max_length,) ))
model.add(LSTM(lstm_output_size))
model.add(Dense(2, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

In [ ]:
import os
import datetime
import time

start = time.time()
history = model.fit(train_data, train_label,
                    epochs=7,
                    batch_size=32,
                    validation_data=(test_data, test_label)
                   )

end = time.time()
print("Model took {:2.0f} seconds to train".format((end - start)))

## Model Evaluation

In [ ]:
_for key in history.history.keys():
    print(key)

In [ ]:
plot_model_history(history)

In [ ]:
test_loss, test_acc = model.evaluate(test_data, test_label)
print('Test accuracy:', test_acc)

In [ ]:
prediction = model.predict(test_data)
prediction.shape

In [ ]:
prediction[1]

In [ ]:
prediction[1][0]

In [ ]:
test_label[i].tolist()

In [ ]:
type(test_data_)

In [ ]:
test_data_ = test_data_.reset_index(drop=True)

In [ ]:
test_data_.head()

In [ ]:
for i in range(10):
  print('%s => %s => expected %s' % (test_data_[i], prediction[i].tolist(), test_label[i].tolist()))

## Wrapping

In [ ]:
def get_sentiment(text):
  the_text = []
  the_text.append(text)
  text_enc = tokenizer.texts_to_sequences(the_text)
  text_enc = pad_sequences(text_enc, maxlen=100)

  prediction = model.predict(text_enc)
  print('%s => positif: %s \t negatif: %s' % (the_text[0], prediction[0][0], prediction[0][1]))


In [ ]:
get_sentiment("telkomsel sinyal bagus")